
Exercises:
- E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?
- E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
- E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
- E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels - wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
- E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?
- E06: meta-exercise! Think of a fun/interesting exercise and complete it.


### Tri-gram model

In [1]:
from micrograd import MLP
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [2]:
words = open("names.txt", "r").read().splitlines()

In [3]:
for j in range(len(words[:1])):
    if len(words) >= 3:
        temp = '.' + words[j] + '.'
        for i in range(len(temp)-2):
            print("prev:", temp[i:i+2])
            print("next:", temp[i+2:i+3])

prev: .e
next: m
prev: em
next: m
prev: mm
next: a
prev: ma
next: .


In [4]:
char = sorted(list(set(''.join(words))))
char = ['.'] + char
char

['.',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [8]:
stoi = {}
for i in range(len(char)):
    stoi[char[i]] = i
stoi

{'.': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [9]:
itoi = {}
for i in range(len(char)):
    itoi[i] = char[i]
itoi

{0: '.',
 1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z'}

In [10]:
for j in range(len(words[:1])):
    if len(words) >= 3:
        words[j] = '.' + words[j] + '.'
        for i in range(len(words[j])-2):
            prev = list(words[j][i:i+2])
            after = words[j][i+2:i+3]
            print("prev:", stoi[prev[0]], stoi[prev[1]])
            print("next:",stoi[after])

prev: 0 0
next: 5
prev: 0 5
next: 13
prev: 5 13
next: 13
prev: 13 13
next: 1
prev: 13 1
next: 0
prev: 1 0
next: 0


In [11]:
# reset dataset
words = open("names.txt", "r").read().splitlines()

In [12]:

xs = []
ys = []

for j in range(len(words)):
    if len(words) >= 3:
        words[j] = '.' + words[j] + '.'
        for i in range(len(words[j])-2):
            prev = list(words[j][i:i+2])
            after = words[j][i+2:i+3]
            prev_i = [stoi[prev[0]], stoi[prev[1]]]
            after_i = stoi[after]
            xs.append(prev_i)
            ys.append(after_i)

In [13]:
xs[0]

[0, 5]

In [14]:
ys[0]

13

In [15]:
xs = torch.tensor(xs)
ys = torch.tensor(ys)
xs.shape, ys.shape

(torch.Size([196113, 2]), torch.Size([196113]))

In [13]:
xs = F.one_hot(xs, num_classes=27).float()
ys = F.one_hot(ys, num_classes=27).float()

In [14]:
xs[0]

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [15]:
ys[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
xs_combined = []
for i in range(xs.shape[0]):
    xs_combined.append(torch.cat((xs[i][0], xs[i][1]), axis=0))

In [17]:
xs_combined = torch.stack(xs_combined)

In [18]:
print(xs_combined.shape)
print(xs_combined[0])

torch.Size([196113, 54])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [19]:
ys[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
xs_combined.shape, ys.shape

(torch.Size([196113, 54]), torch.Size([196113, 27]))

In [21]:
nn = MLP(27+27,[256,128,27])

In [22]:
output = nn(xs_combined[:10], activation="softmax")

In [23]:
print("output shape:", len(output), len(output[0]), "\n")
print(len(output[0]), output[0])
print("\n",sum(output[0]))

output shape: 10 27 

27 [Value(data=0.017341060331362762, grad=0.0), Value(data=0.09557937436245291, grad=0.0), Value(data=0.01487120397915873, grad=0.0), Value(data=0.01301352559069451, grad=0.0), Value(data=0.09558086968468868, grad=0.0), Value(data=0.095581347131435, grad=0.0), Value(data=0.012936526729709756, grad=0.0), Value(data=0.09558217843524486, grad=0.0), Value(data=0.012941480178322216, grad=0.0), Value(data=0.013129272745706369, grad=0.0), Value(data=0.012936456415162507, grad=0.0), Value(data=0.015228516958556844, grad=0.0), Value(data=0.01293644168760002, grad=0.0), Value(data=0.03747530167610472, grad=0.0), Value(data=0.09547584585100993, grad=0.0), Value(data=0.013209223296394764, grad=0.0), Value(data=0.012950578892912826, grad=0.0), Value(data=0.013149379772021762, grad=0.0), Value(data=0.012940675086374292, grad=0.0), Value(data=0.09558773515133952, grad=0.0), Value(data=0.012936883294326117, grad=0.0), Value(data=0.012940214964484947, grad=0.0), Value(data=0.01294

In [24]:
loss = nn.cross_entropy_loss(output, ys[:10])
print("loss:",loss)

loss: Value(data=3.942078113555908, grad=0.0)


##### E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

In [25]:
# reset dataset

print("dataset x:", xs.shape)
print("dataset y:",ys.shape)

train_percent = int((.8 * xs.shape[0]))
train_set_xs = xs[:train_percent]

rest_xs = xs[train_percent+1:]
dev_set_xs, test_set_xs = rest_xs[:len(rest_xs)//2], rest_xs[len(rest_xs)//2+1:]

print("\ntrain_set_xs: ", train_set_xs.shape)
print("dev_set_xs: ", dev_set_xs.shape)
print("test_set_xs: ", test_set_xs.shape)

train_set_ys = ys[:train_percent]

rest_ys = ys[train_percent+1:]
dev_set_ys, test_set_ys = rest_ys[:len(rest_ys)//2], rest_ys[len(rest_ys)//2+1:]

print("\ntrain_set_ys: ", train_set_ys.shape)
print("dev_set_ys: ", dev_set_ys.shape)
print("test_set_ys: ", test_set_ys.shape)

dataset x: torch.Size([196113, 2, 27])
dataset y: torch.Size([196113, 27])

train_set_xs:  torch.Size([156890, 2, 27])
dev_set_xs:  torch.Size([19611, 2, 27])
test_set_xs:  torch.Size([19610, 2, 27])

train_set_ys:  torch.Size([156890, 27])
dev_set_ys:  torch.Size([19611, 27])
test_set_ys:  torch.Size([19610, 27])


In [26]:
nn = MLP(27+27,[27])

In [27]:
lr = 0.01
epochs = 10

def l1_reg(x):
    loss = 0
    for i in range(len(x)):
        loss += x[i]**2

    # mean
    return loss/len(x)
    
for epoch in range(epochs):

    logits = nn(xs_combined[:10], activation="softmax")
    reg_loss = sum([0.01 * l1_reg(logit) for logit in logits]) / len(logits)
    print("reguralization loss:", reg_loss.data)
    loss = nn.cross_entropy_loss(logits, ys[:10])

    loss += reg_loss
    
    print("loss: ", loss.data)
    
    # zero grad
    for p in nn.parameters():
        p.grad = 0.0

    loss.backward() 
    for p in nn.parameters():
        p.data += -lr * p.grad


reguralization loss: 1.9588800988490805e-05
loss:  tensor(3.1066)
reguralization loss: 1.958823872295276e-05
loss:  tensor(3.1048)
reguralization loss: 1.958768367064149e-05
loss:  tensor(3.1031)
reguralization loss: 1.9587135839633298e-05
loss:  tensor(3.1014)
reguralization loss: 1.9586595250911955e-05
loss:  tensor(3.0997)
reguralization loss: 1.9586062006470554e-05
loss:  tensor(3.0979)
reguralization loss: 1.958553616079376e-05
loss:  tensor(3.0962)
reguralization loss: 1.958501772538729e-05
loss:  tensor(3.0945)
reguralization loss: 1.958450678249514e-05
loss:  tensor(3.0928)
reguralization loss: 1.95840032882625e-05
loss:  tensor(3.0911)


In [28]:
import numpy as np

def get_char_from_one_hot(one_hot):
    for i in range(len(one_hot)):
        if one_hot[i] == 1:
            return itoi[i]

In [29]:

for i in range(10):
    
    input = [torch.cat((test_set_xs[i][0], test_set_xs[i][1]), dim=0)]
    logits = nn(input, activation="softmax")
    logits = logits[0] 
    max = logits[0].data
    index = 0
    for j in range(len(logits)):
        if logits[j].data > max:
            max = logits[j].data
            index = j
    
    print("characters:", get_char_from_one_hot(test_set_xs[i][0]), get_char_from_one_hot(test_set_xs[i][1]))
    print("next char prediction:", itoi[index])
    print("actual answer:", get_char_from_one_hot(test_set_ys[i]))
    print("\n")

characters: n i
next char prediction: q
actual answer: k


characters: i k
next char prediction: q
actual answer: i


characters: k i
next char prediction: q
actual answer: n


characters: i n
next char prediction: m
actual answer: .


characters: . a
next char prediction: a
actual answer: n


characters: a n
next char prediction: i
actual answer: i


characters: n i
next char prediction: q
actual answer: l


characters: i l
next char prediction: q
actual answer: .


characters: . a
next char prediction: a
actual answer: n


characters: a n
next char prediction: i
actual answer: n




In [30]:
# 54 input -> 1 neuron -> 1 output
W = torch.randn((54,1))

train_set_combined = torch.cat((train_set_xs[0][0],train_set_xs[0][1]), axis=0)

print("input shape:",train_set_combined.shape)
print(train_set_combined)
print("\n")

print("layer shape:", W.shape)
#print("layer:", W)
print("\n")
forward = torch.matmul(train_set_combined, W)
print("output shape -> input * W:",forward.shape)
print(forward)
# Single neuron of 54 weights

input shape: torch.Size([54])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


layer shape: torch.Size([54, 1])


output shape -> input * W: torch.Size([1])
tensor([0.2171])


In [31]:
# 54 input -> 27 NEURONS -> 27 output vector
W = torch.randn((54,27))

train_set_combined = [torch.cat((x[0],x[1]), axis=0) for x in train_set_xs[:1]]
train_set_combined = torch.stack(train_set_combined)

In [32]:
print("input shape:",train_set_combined.shape)
print(train_set_combined)
print("\n")

print("layer shape:", W.shape)
print("layer:", W)
print("\n")
forward = torch.matmul(train_set_combined, W)
print("output shape -> input * W:",forward.shape)
print(forward)
# Single

input shape: torch.Size([1, 54])
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


layer shape: torch.Size([54, 27])
layer: tensor([[-1.8492,  2.9028,  0.4132,  ..., -0.5752,  1.4118, -0.4575],
        [-1.6327, -2.5746,  0.1141,  ..., -0.2463,  1.5246,  0.1670],
        [ 0.1483,  0.1914,  0.8098,  ..., -0.5611,  0.4704, -0.2269],
        ...,
        [ 1.2092,  0.4162, -1.0370,  ..., -0.1952, -0.0832,  0.8125],
        [-0.9336,  0.5319, -0.6115,  ...,  0.0420, -1.3682,  0.4505],
        [-0.0941,  0.5582,  1.3776,  ...,  0.5271, -0.4520, -0.1664]])


output shape -> input * W: torch.Size([1, 27])
tensor([[-3.0633,  3.2625,  0.2344,  0.8663, -2.7010,  1.8308, -1.5601,  2.6567,
          0.7999,  0.5840, -0.0600,  0.1228, -0.4268, -0.5907, -1.7441, -0.0149,
         -0.6291,  3.5341, -0.2444

In [33]:
train_set_combined = [torch.cat((x[0],x[1]), axis=0) for x in train_set_xs[:100]]
train_set_combined = torch.stack(train_set_combined)
print("input shape:",train_set_combined.shape)
print(train_set_combined)
print("\n")

print("layer shape:", W.shape)
print("layer:", W)
print("\n")
forward = torch.matmul(train_set_combined, W)
print("output shape -> input * W:",forward.shape)
print(forward)
# Doesn't matter if there are multiple inputs, output will be (no of inputs, neuron output size)

input shape: torch.Size([100, 54])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


layer shape: torch.Size([54, 27])
layer: tensor([[-1.8492,  2.9028,  0.4132,  ..., -0.5752,  1.4118, -0.4575],
        [-1.6327, -2.5746,  0.1141,  ..., -0.2463,  1.5246,  0.1670],
        [ 0.1483,  0.1914,  0.8098,  ..., -0.5611,  0.4704, -0.2269],
        ...,
        [ 1.2092,  0.4162, -1.0370,  ..., -0.1952, -0.0832,  0.8125],
        [-0.9336,  0.5319, -0.6115,  ...,  0.0420, -1.3682,  0.4505],
        [-0.0941,  0.5582,  1.3776,  ...,  0.5271, -0.4520, -0.1664]])


output shape -> input * W: torch.Size([100, 27])
tensor([[-3.0633,  3.2625,  0.2344,  ..., -0.4990,  1.3015, -0.9433],
        [ 0.4810,  1.6669, -0.2411,  ..., -1.1793, -1.5212, -2.1711],
        [ 2.0463,  0.8799, -0.5501,  ..., -

In [34]:
def softmax(x):
    
    counts = [logit.exp() for logit in x]
    denominator = sum(counts)
    out = [c / denominator for c in counts]
    
    return out

softmax_layer = []
for i in range(len(forward)):
    softmax_layer.append(softmax(forward[i]))

In [35]:
print(torch.tensor(softmax_layer[0]))
print(sum(softmax_layer[0]))
print(len(softmax_layer[0]))

tensor([0.0004, 0.2444, 0.0118, 0.0223, 0.0006, 0.0584, 0.0020, 0.1334, 0.0208,
        0.0168, 0.0088, 0.0106, 0.0061, 0.0052, 0.0016, 0.0092, 0.0050, 0.3207,
        0.0073, 0.0354, 0.0154, 0.0159, 0.0032, 0.0010, 0.0057, 0.0344, 0.0036])
tensor(1.)
27


In [36]:
# 100 of 54 vector input -> 54 of 128 neurons -> 100 of 128 output vector
# 100 of 128 vector input -> 128 of 64 neurons -> 100 of 64 output vector 
# 100 of 64 vector input -> 64 of 27 neurons -> 100 of 27 output vector 

layer_1 = torch.randn((54,128)) # Layer(54,128)
layer_2 = torch.randn((128,64)) # Layer(128, 64)
layer_3 = torch.randn((64,27)) # Layer(64,27)

layer_1_output = torch.matmul(train_set_combined,layer_1)
layer_2_output = torch.matmul(layer_1_output,layer_2)
layer_3_output = torch.matmul(layer_2_output,layer_3)

In [37]:
print(layer_1_output.shape)
print(layer_2_output.shape)
print(layer_3_output.shape)

torch.Size([100, 128])
torch.Size([100, 64])
torch.Size([100, 27])


##### E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels - wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

In [38]:
# Since it just selects the row of weight matrix, basically if one hot vector is 1 at 4th index row wise, it is practically returning
# the 4th row of weight matrix(layer matrix), since all the 0s multiplying with other items don't count, we end just multiplying
# 1 * 4th number of each neuron(column)

one_hot_27 = train_set_combined[0][:27]
layer_27_neuron = torch.randn((27,27))

print(one_hot_27)

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [39]:
forward = torch.matmul(one_hot_27, layer_27_neuron)
layer_row_0th = layer_27_neuron[0] # 0th index at one hot is 1

print(forward)
print(layer_row_0th)

tensor([-0.3805, -2.4144, -0.3294, -0.1598, -0.3500,  2.4460, -0.2325,  0.2086,
        -1.2851, -1.2860,  0.8684,  1.8610, -0.7826, -0.6321, -0.7637, -1.0874,
        -0.8241, -0.5835, -1.6526, -0.7508,  0.1022, -0.0134, -1.4359,  0.7837,
        -1.7791, -1.3529,  0.2274])
tensor([-0.3805, -2.4144, -0.3294, -0.1598, -0.3500,  2.4460, -0.2325,  0.2086,
        -1.2851, -1.2860,  0.8684,  1.8610, -0.7826, -0.6321, -0.7637, -1.0874,
        -0.8241, -0.5835, -1.6526, -0.7508,  0.1022, -0.0134, -1.4359,  0.7837,
        -1.7791, -1.3529,  0.2274])


In [40]:
forward == layer_row_0th

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True])

In [41]:
layer_27 = torch.randn((54,27), requires_grad=True)

In [42]:
sample_train = xs_combined[:50]
sample_label = test_set_ys[:50]

In [43]:
logits = torch.matmul(sample_train, layer_27)
print(logits.shape)

torch.Size([50, 27])


In [44]:
def softmax(x):
    
    counts = [logit.exp() for logit in x]
    denominator = sum(counts)
    out = [c / denominator for c in counts]
    
    return torch.tensor(out)
    
prob = [softmax(logit) for logit in logits]
prob = torch.stack(prob)

In [45]:
print(prob.shape)
print(prob[0])
print(sum(prob[0]))

torch.Size([50, 27])
tensor([0.0067, 0.0141, 0.0050, 0.0637, 0.0143, 0.0173, 0.1038, 0.1516, 0.0068,
        0.0020, 0.0054, 0.0058, 0.1516, 0.0220, 0.0203, 0.0098, 0.0040, 0.1206,
        0.0360, 0.1332, 0.0168, 0.0225, 0.0067, 0.0188, 0.0162, 0.0143, 0.0107])
tensor(1.0000)


In [46]:
def cross_entropy_loss(x,y):
    loss = 0
    for i in range(len(x)):
        loss += y[i] * x[i].log()
    return (-loss)
    
loss = sum(cross_entropy_loss(x,y) for x,y in zip(prob, sample_label)) / 50
loss

tensor(4.5414)

##### E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

### training

In [47]:
lr = 0.01
epochs = 10
train_set_combined = [torch.cat((x[0],x[1]), axis=0) for x in train_set_xs[:100]]

def l1_reg(x):
    loss = 0
    for i in range(len(x)):
        loss += x[i]**2

    # mean
    return loss/len(x)
    
for epoch in range(epochs):

    logits = nn(train_set_combined[:10], activation="softmax")
    reg_loss = sum([0.2 * l1_reg(logit) for logit in logits]) / len(logits)
    print("reguralization loss:", reg_loss.data)
    loss = nn.cross_entropy_loss(logits, train_set_ys[:10])

    loss += reg_loss
    
    print("loss: ", loss.data)
    
    # zero grad
    for p in nn.parameters():
        p.grad = 0.0

    loss.backward() 
    for p in nn.parameters():
        p.data += -lr * p.grad


reguralization loss: 0.00039167014790667063
loss:  tensor(3.0898)
reguralization loss: 0.0003916603772154336
loss:  tensor(3.0881)
reguralization loss: 0.00039165075834146005
loss:  tensor(3.0864)
reguralization loss: 0.00039164129305505605
loss:  tensor(3.0847)
reguralization loss: 0.0003916319808647537
loss:  tensor(3.0831)
reguralization loss: 0.00039162282318687343
loss:  tensor(3.0814)
reguralization loss: 0.00039161382139936874
loss:  tensor(3.0797)
reguralization loss: 0.00039160497477526754
loss:  tensor(3.0781)
reguralization loss: 0.0003915962844063818
loss:  tensor(3.0764)
reguralization loss: 0.000391587750374981
loss:  tensor(3.0747)


### Testing

In [48]:
test_set_combined = [torch.cat((x[0],x[1]), axis=0) for x in test_set_xs[:100]]
    
logits = nn(test_set_combined[:10], activation="softmax")
loss = nn.cross_entropy_loss(logits, test_set_ys[:10])
print("loss: ", loss.data)


loss:  tensor(3.6262)


##### Decreasing importance of regularization by reduce paramater results in increased training accuracy but no difference in testing accuracy
##### Increaseing regularization parameters resulting in about the same training accuracy but higher accuracy on testing, showcasing more generalization and better model